### CSV and Excel Files -Structured Data

In [1]:
import pandas as pd
import os
import openpyxl

In [2]:
os.makedirs("data/structured_files",exist_ok=True)

In [3]:
data = {
    'Product': ['Laptop', 'Mouse', 'Keyboard', 'Monitor', 'Webcam'],
    'Category': ['Electronics', 'Accessories', 'Accessories', 'Electronics', 'Electronics'],
    'Price': [999.99, 29.99, 79.99, 299.99, 89.99],
    'Stock': [50, 200, 150, 75, 100],
    'Description': [
        'High-performance laptop with 16GB RAM and 512GB SSD',
        'Wireless optical mouse with ergonomic design',
        'Mechanical keyboard with RGB backlighting',
        '27-inch 4K monitor with HDR support',
        '1080p webcam with noise cancellation'
    ]
}

df = pd.DataFrame(data)
df.to_csv("data/structured_files/products.csv", index=False)


In [4]:
# Save as Excel with multiple sheets
with pd.ExcelWriter("data/structured_files/products.xlsx") as writer:
    df.to_excel(writer, sheet_name="Products", index=False)
    summary_data ={
        'Category' : ['Electronics', 'Accessories'],
        'Total_items' : [3,2],
        'Total_Value' : [1389.97, 109.98]
    }
    pd.DataFrame(summary_data).to_excel(writer, sheet_name="Summary", index=False)

### CSV Processing

In [5]:
from langchain_community.document_loaders import CSVLoader, UnstructuredCSVLoader

In [6]:
#Method 1 : CSVLoader - Each row becomes a document
print("1 CSVloader - Row-based Documents")
csv_loader = CSVLoader(
    file_path='data/structured_files/products.csv',
    encoding ='utf-8',
    csv_args={
        'delimiter': ',',
        'quotechar': '"',
    }
)
csv_docs = csv_loader.load()
print(f"Loaded {len(csv_docs)} documents (one per row)")
print("\nFirst document :")
print(f"Content : {csv_docs[0].page_content}")
print(f"Metadata : {csv_docs[0].metadata}")


1 CSVloader - Row-based Documents
Loaded 5 documents (one per row)

First document :
Content : Product: Laptop
Category: Electronics
Price: 999.99
Stock: 50
Description: High-performance laptop with 16GB RAM and 512GB SSD
Metadata : {'source': 'data/structured_files/products.csv', 'row': 0}


In [12]:
# Method 2 Custom CSV Processing for better control
from typing import List
from langchain_core.documents import Document
print("\n 2 Custom CSV Processing ")
def process_csv_intelligently(filepath:str) -> List[Document]:
   df = pd.read_csv(filepath)
   documents =[]

   #Strategy 1: One Document per row with structured content
   for idx, row in df.iterrows():
      # Create structured content
      content = f"""Product Information:
      Name : {row['Product']}
      Category : {row['Category']}
      Price : ${row['Price']}
      Stock : {row['Stock']} units
      Description: {row['Description']}"""
      doc = Document(
            page_content=content,
            metadata={
               'source' : filepath,
               'row_index' : idx,
               'product_name': row['Product'],
                'category': row['Category'],
                'price': row['Price'],
                'data_type': 'product_info'
            }
        )
      documents.append(doc)
      return documents


 2 Custom CSV Processing 


In [13]:
process_csv_intelligently("data/structured_files/products.csv")

[Document(metadata={'source': 'data/structured_files/products.csv', 'row_index': 0, 'product_name': 'Laptop', 'category': 'Electronics', 'price': 999.99, 'data_type': 'product_info'}, page_content='Product Information:\n      Name : Laptop\n      Category : Electronics\n      Price : $999.99\n      Stock : 50 units\n      Description: High-performance laptop with 16GB RAM and 512GB SSD')]

In [14]:
# 📊 CSV Processing Strategies
print("\n📊 CSV Processing Strategies:")
print("\n1. Row-based (CSVLoader):")
print("  ✅ Simple one-row-one-document")
print("  ✅ Good for record lookups")
print("  ❌ Loses table context")

print("\n2. Intelligent Processing:")
print("  ✅ Preserves relationships")
print("  ✅ Creates summaries")
print("  ✅ Rich metadata")
print("  ✅ Better for Q&A")


📊 CSV Processing Strategies:

1. Row-based (CSVLoader):
  ✅ Simple one-row-one-document
  ✅ Good for record lookups
  ❌ Loses table context

2. Intelligent Processing:
  ✅ Preserves relationships
  ✅ Creates summaries
  ✅ Rich metadata
  ✅ Better for Q&A


In [16]:
# Method 1: Using pandas for full control
print("1️⃣ Pandas-based Excel Processing")
def process_excel_with_pandas(filepath: str) -> List[Document]:
    """Process Excel with sheet awareness"""
    documents = []
    
    # Read all sheets
    excel_file = pd.ExcelFile(filepath)
    
    for sheet_name in excel_file.sheet_names:
        df = pd.read_excel(filepath, sheet_name=sheet_name)
        
        # Create document for each sheet
        sheet_content = f"Sheet: {sheet_name}\n"
        sheet_content += f"Columns: {', '.join(df.columns)}\n"
        sheet_content += f"Rows: {len(df)}\n\n"
        sheet_content += df.to_string(index=False)
        
        doc = Document(
            page_content=sheet_content,
            metadata={
                'source': filepath,
                'sheet_name': sheet_name,
                'num_rows': len(df),
                'num_columns': len(df.columns),
                'data_type': 'excel_sheet'
            }
        )
        documents.append(doc)
    
    return documents

1️⃣ Pandas-based Excel Processing


In [17]:
excel_docs = process_excel_with_pandas('data/structured_files/inventory.xlsx')
print(f"Processed {len(excel_docs)} sheets")

Processed 2 sheets


In [18]:
from langchain_community.document_loaders import UnstructuredExcelLoader
# Method 2: UnstructuredExcelLoader
print("\n2️⃣ UnstructuredExcelLoader")
try:
    excel_loader = UnstructuredExcelLoader(
        'data/structured_files/inventory.xlsx',
        mode="elements"
    )
    unstructured_docs = excel_loader.load()
    print("  ✅ Handles complex Excel features")
    print("  ✅ Preserves formatting info")
    print("  ❌ Requires unstructured library")
except Exception as e:
    print("  ℹ️ Requires unstructured library with Excel support")


2️⃣ UnstructuredExcelLoader
  ℹ️ Requires unstructured library with Excel support
